In [10]:
# Importar librerías 

import pandas as pd  

from bs4 import BeautifulSoup
import requests

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from time import sleep  

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import re


In [11]:
class ExtraerDatosActoresSelenium:
    def __init__(self, archivo_csv): 
        self.archivo_csv = pd.read_csv(archivo_csv)  
        self.info_actores = [] 
        self.info_actores_tupla = [] 
        
    def pag_principal_google(self):
            # Inicializar el driver de Chrome
            options = webdriver.ChromeOptions()
            options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
            self.driver = webdriver.Chrome(options=options)
            sleep(2)
            self.driver.maximize_window()
            self.driver.get("https://www.google.es/")
            # acepta cookies
            sleep(5)
            try:
                self.driver.find_element("css selector", "#L2AGLb").click()
            except:
                pass
            self.escoger_actor_csv()          

    def escoger_actor_csv(self):
        # Itera por cada pelicula del csv
        for i in range(50, 70):  
            # Escoge info del csv para la busqueda  
            actor = (f"{self.archivo_csv['actor'][i]}")  
            # Mete en la lista el actor  
            self.info_actores.append(actor)    
            # Busca info del actor      
            self.busqueda_google("imdb", actor)                     
            self.busqueda_google("imdb premios", actor)
            # Si no tiene premios, apendea vacio
            if len(self.info_actores) == 4:
                self.info_actores.append("")
            # Si la lista esta completa apendea la tupla
            if len(self.info_actores) > 4:
                tupla = tuple(self.info_actores)
                print(tupla)
                self.info_actores_tupla.append(tupla)
            # resetea la lista
            self.info_actores.clear()  
            # Guarda en archivo
            self.guardar_info_actores_csv("info_actores")
       
    def busqueda_google(self, pagina, dato_buscador):
        sleep(3)
        # Ingresar el título y año de la película en el campo de búsqueda
        self.driver.find_element("css selector", "#APjFqb").clear()
        self.driver.find_element("css selector", "#APjFqb").send_keys(f'{pagina} "{dato_buscador}"', Keys.ENTER)             
       
        # Definimos la url de la página de la que vamos a sacar datos y llama a la api   
        try:            
            sleep(5)            
            href = self.driver.find_element("css selector", "#rso .yuRUbf > div > span > a").get_attribute("href")   
            # si el primer resultado de google no es de imdb pasa a la siguiente pelicula 
            if "imdb" in href:                                
                self.llamada_api(href) 
            else:
                self.info_actores.append("")
                print(f"ha fallado la llamada a la pagina {pagina}")
                
        except:
            pass    
    
    def llamada_api (self, href):

        # Definimos un User-Agent para que la solicitud parezca venir de un navegador real        
        
        headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
              
        # Hacemos la request a la página de la que queremos sacar la info (imbd)
        resultado = requests.get(href, headers=headers)

        # Vemos si todo ha ido bien
        print("La respuesta de la petición es:", resultado.status_code)

        # creamos el objeto BeautifulSoup para poder acceder al contenido solicitado
        sopa = BeautifulSoup(resultado.content, 'html.parser')
        # si el objeto viene de imdb premios llamamos a limpiar datos de premios
        if resultado.status_code == 200:
            if "award" in href: 
                self.limpieza_datos_premios_imdb(sopa)     
            # si el objeto viene de imdb llamamos a limpiar datos actor        
            else : 
                self.limpieza_datos_actores_imdb(sopa)          
        else:
            return         
    
    def limpieza_datos_actores_imdb(self, sopa):
        if len(self.info_actores) == 1 :  
            # sacamos año nacimiento        
            anho_nacimiento = sopa.find_all("span", {"class": "sc-59a43f1c-2"})
            self.info_actores.append(anho_nacimiento[1].text)
            
            # pq es conocido
            conocido = sopa.find_all("a", {"class": "ipc-metadata-list-summary-item__t"})
            lista =[]
            for i in range(0,10):   
                lista.append(conocido[i].text)
            self.info_actores.append(lista)

            # que hace
            que_hace = sopa.find_all("ul", {"class": "ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt"})
            self.info_actores.append(que_hace[0].text)
        else:
            return

    def limpieza_datos_premios_imdb(self, sopa):
        premios = []
        # premios
        premios_sopa = sopa.find_all("a", {"class": "ipc-metadata-list-summary-item__t"})
        for i in range(0,4):
            premios.append(premios_sopa[i].text)
        self.info_actores.append(premios)

    def guardar_info_actores_csv (self, nombre_archivo):        
        try:
            df_actores = pd.DataFrame(self.info_actores_tupla, columns=["nombre_apellido", "anho_nacimiento", "peliculas", "que_hace", "premios"])
            df_actores.to_csv(nombre_archivo+".csv", index=False, header=True)
        except:
            print("no puedo convertir a csv")
    

       

In [12]:
csv = "info_peliculas_Drama_actores.csv"
# csv = input("Introduce el género del que quieres conseguir la información")
try:
    peliculas_accion_sele = ExtraerDatosActoresSelenium(csv)
    peliculas_accion_sele.pag_principal_google()
except:
    print("Hay algún error, comprueba que exista el fichero")
    # pass


La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
('Mark Gantt', 'December 10', ['Soul Mates', 'Un embarazo peligroso', 'Una madre ejemplar', 'Seamlessly She', 'Intricate Vengeance', 'Asesinato en México', 'Donor', 'Suite 7', 'Martine', 'Netherworld'], 'DirectorWriterProducer', '')
La respuesta de la petición es: 200
La respuesta de la petición es: 200
('Corbin Timbrook', 'June 1958', ['Bail Out', 'Specter', 'Emails from Heaven', 'Stepmom from Hell', 'The Twisters', "Pregnant by my Sister's Fiancé", "Becoming My Ex's Mother-In-Law", 'Mating of the Twilight Goddess', 'Love, Courage and the Battle of Bushy Run', 'After Divorce, Stripper CEO and I Dominate Hollywood'], 'ActorProducerDirector', '')
La respuesta de la petición es: 200
La respuesta de la petición es: 200
('John Brotherton', 'August 21, 1980', ['9-1-1', 'Los detectives muertos', 'Chosen Family', "The Undertaker's Wife", 'Christmas on